# Data Engineer Certification - Practical Exam - Supplement Experiments

1001-Experiments makes personalized supplements tailored to individual health needs.

1001-Experiments aims to enhance personal health by using data from wearable devices and health apps.

This data, combined with user feedback and habits, is used to analyze and refine the effectiveness of the supplements provided to the user through multiple small experiments.

The data engineering team at 1001-Experiments plays a crucial role in ensuring the collected health and activity data from thousands of users is accurately organized and integrated with the data from supplement usage. 

This integration helps 1001-Experiments provide more targeted health and wellness recommendations and improve supplement formulations.


## Task

1001-Experiments currently has the following four datasets with four months of data:
 - "user_health_data.csv" which logs daily health metrics, habits and data from wearable devices,
 - "supplement_usage.csv" which records details on supplement intake per user,
 - "experiments.csv" which contains metadata on experiments, and
 - "user_profiles.csv" which contains demographic and contact information of the users.

Each dataset contains unique identifiers for users and/or their supplement regimen.

The developers and data scientsits currently manage code that cross-references all of these data sources separately, which is cumbersome and error-prone.

Your manager has asked you to write a Python function that cleans and merges these datasets into a single dataset.

The final dataset should provide a comprehensive view of each user's health metrics, supplement usage, and demographic information.

- To test your code, your manager will run only the code `merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')`
- Your `merge_all_data` function must return a DataFrame, with columns as described below.
- All columns must accurately match the descriptions provided below, including names.


## Data

The provided data is structured as follows:

![database schema](schema.png)

The function you write should return data as described below.

There should be a unique row for each daily entry combining health metrics and supplement usage.

Where missing values are permitted, they should be in the default Python format unless stated otherwise.

| Column Name        | Description |
|--------------------|-------------|
| user_id            | Unique identifier for each user. </br>There should not be any missing values. |
| date               | The date the health data was recorded or the supplement was taken, in date format. </br>There should not be any missing values. |
| email              | Contact email of the user. </br>There should not be any missing values. |
| user_age_group  | The age group of the user, one of: 'Under 18', '18-25', '26-35', '36-45', '46-55', '56-65', 'Over 65' or 'Unknown' where the age is missing.|
| experiment_name    | Name of the experiment associated with the supplement usage. </br>Missing values for users that have user health data only is permitted. |
| supplement_name    | The name of the supplement taken on that day. Multiple entries are permitted. </br>Days without supplement intake should be encoded as 'No intake'. |
| dosage_grams       | The dosage of the supplement taken in grams. Where the dosage is recorded in mg it should be converted by division by 1000.</br>Missing values for days without supplement intake are permitted. |
| is_placebo         | Indicator if the supplement was a placebo (true/false). </br>Missing values for days without supplement intake are permitted. |
| average_heart_rate | Average heart rate as recorded by the wearable device. </br>Missing values are permitted. |
| average_glucose    | Average glucose levels as recorded on the wearable device. </br>Missing values are permitted. |
| sleep_hours        | Total sleep in hours for the night preceding the current day’s log. </br>Missing values are permitted. |
| activity_level     | Activity level score between 0-100. </br>Missing values are permitted. |

In [49]:
import pandas as pd
import numpy as np

In [50]:
import pandas as pd
import numpy as np

def merge_all_data(user_health_data_path, supplement_usage_path, experiments_path, user_profiles_path):
    # 1. Load the four datasets
    df_health = pd.read_csv(user_health_data_path)
    df_usage = pd.read_csv(supplement_usage_path)
    df_experiments = pd.read_csv(experiments_path)
    df_profiles = pd.read_csv(user_profiles_path)

    # 2. Convert date columns to date format
    # Standardizing format to ensure proper merging
    df_health['date'] = pd.to_datetime(df_health['date']).dt.normalize()
    df_usage['date'] = pd.to_datetime(df_usage['date']).dt.normalize()

    # 3. Clean categorical/text data by manipulating strings
    # Removing 'h'/'H' from sleep_hours and converting to float
    if df_health['sleep_hours'].dtype == 'object':
        df_health['sleep_hours'] = (
            df_health['sleep_hours']
            .str.replace(r'[hH]', '', regex=True)
            .astype(float)
        )

    # 4. Convert values between data types: dosage_grams
    # Division by 1000 if recorded in mg
    df_usage['dosage_grams'] = np.where(
        df_usage['dosage_unit'].str.lower() == 'mg',
        df_usage['dosage'] / 1000,
        df_usage['dosage']
    )

    # 5. Define age groups as per strict descriptions
    def categorize_age(age):
        if pd.isna(age): return 'Unknown'
        if age < 18: return 'Under 18'
        elif 18 <= age <= 25: return '18-25'
        elif 26 <= age <= 35: return '26-35'
        elif 36 <= age <= 45: return '36-45'
        elif 46 <= age <= 55: return '46-55'
        elif 56 <= age <= 65: return '56-65'
        else: return 'Over 65'

    df_profiles['user_age_group'] = df_profiles['age'].apply(categorize_age)

    # 6. Combine multiple tables (Outer join between metrics and supplements)
    # This ensures entries exist for days with activity but no supplement
    merged_df = pd.merge(df_health, df_usage, on=['user_id', 'date'], how='outer')

    # Merge Experiment names
    merged_df = pd.merge(
        merged_df, 
        df_experiments[['experiment_id', 'name']], 
        on='experiment_id', 
        how='left'
    ).rename(columns={'name': 'experiment_name'})

    # Merge User Demographic Profiles
    merged_df = pd.merge(
        merged_df, 
        df_profiles[['user_id', 'email', 'user_age_group']], 
        on='user_id', 
        how='left'
    )

    # 7. Identify and replace missing values
    # Encode days without supplement intake as 'No intake'
    merged_df['supplement_name'] = merged_df['supplement_name'].fillna('No intake')
    
    # Enforce NO MISSING VALUES for user_id, date, and email
    merged_df = merged_df.dropna(subset=['user_id', 'date', 'email'])

    # 8. Final Column Selection and Ordering (Must match requirement table)
    required_columns = [
        'user_id', 'date', 'email', 'user_age_group', 'experiment_name',
        'supplement_name', 'dosage_grams', 'is_placebo',
        'average_heart_rate', 'average_glucose', 'sleep_hours', 'activity_level'
    ]
    
    return merged_df[required_columns]

In [57]:
result_df=merge_all_data('user_health_data.csv', 'supplement_usage.csv', 'experiments.csv', 'user_profiles.csv')
result_df.head()

,user_id,date,email,user_age_group,experiment_name,supplement_name,dosage_grams,is_placebo,average_heart_rate,average_glucose,sleep_hours,activity_level
0,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-01-31,hi_1@example.com,Over 65,Sleep Quality,Zinc,0.149315,False,93.055612,70.089910,8.8,1
1,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-02-28,hi_1@example.com,Over 65,NaN,No intake,NaN,NaN,88.059964,78.411148,8.0,3
2,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-03-31,hi_1@example.com,Over 65,Endurance,Magnesium,0.239949,True,78.373746,107.418818,11.9,1
3,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-03-31,hi_1@example.com,Over 65,Focus,Placebo,0.416022,False,78.373746,107.418818,11.9,1
4,c6ae338a-9f95-481c-a88d-24a58bc8fc71,2018-04-30,hi_1@example.com,Over 65,Recovery,Placebo,0.324913,False,62.204061,117.259092,5.1,1
